In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/Shared\ drives/ecl-pedro/video-coding-utils/notebooks/

/content/drive/Shared drives/ecl-pedro/video-coding-utils/notebooks


---

In [7]:
from utils.utils import *
from utils import models

In [8]:
import tensorflow as tf

In [9]:
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.image import psnr
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [11]:
def PSNR(x,y):
    return psnr(x,y,max_val=1)

In [12]:
ds = tf.data.Dataset.from_tensor_slices(list(range(50000)))

In [13]:
ds = ds.shuffle(50000)

In [14]:
def parse_function(xdir, ydir):
    
    def fun(i):
        
        i = tf.strings.as_string(i)
        fcompressed = tf.io.read_file(xdir + i + '.png')
        compressed = tf.image.decode_png(fcompressed, channels=1)
        compressed = tf.image.convert_image_dtype(compressed, tf.float32)
        compressed = tf.image.resize(compressed, [64, 64])
        
        fgroundtruth = tf.io.read_file(ydir + i + '.png')
        groundtruth = tf.image.decode_png(fgroundtruth, channels=1)
        groundtruth = tf.image.convert_image_dtype(groundtruth, tf.float32)
        groundtruth = tf.image.resize(groundtruth, [64, 64])
        
        return compressed, groundtruth
    
    return fun

In [15]:
ds = ds.map(parse_function('../../data/preprocessed/basketball/compressed/', '../../data/preprocessed/basketball/ground_truth/'))

In [16]:
bds = ds.batch(128)

In [17]:
model = models.QECNN_I()

In [18]:
model.build((None, 64, 64, 1))

In [19]:
model.compile(
    optimizer='SGD',
    loss=MSE,
    metrics=[PSNR]
)

In [20]:
ckpt_callback = ModelCheckpoint(
    filepath='checkpoints/QECNN_I',
    save_weights_only=True,
    verbose=0
)

In [22]:
history = model.fit(
    bds,
    epochs=50,
    callbacks=ckpt_callback
)

Epoch 1/50
391/391 [==============================] - 21705s 56s/step - loss: 0.0099 - PSNR: 21.0912
Epoch 2/50
391/391 [==============================] - 107s 274ms/step - loss: 0.0039 - PSNR: 24.6856
Epoch 3/50
391/391 [==============================] - 109s 278ms/step - loss: 0.0029 - PSNR: 25.9843
Epoch 4/50
391/391 [==============================] - 110s 281ms/step - loss: 0.0025 - PSNR: 27.1308
Epoch 5/50
391/391 [==============================] - 110s 282ms/step - loss: 0.0019 - PSNR: 28.2666
Epoch 6/50
391/391 [==============================] - 111s 283ms/step - loss: 0.0017 - PSNR: 28.8590
Epoch 7/50
391/391 [==============================] - 111s 283ms/step - loss: 0.0016 - PSNR: 29.0739
Epoch 8/50
391/391 [==============================] - 111s 283ms/step - loss: 0.0016 - PSNR: 29.0728
Epoch 9/50
391/391 [==============================] - 111s 283ms/step - loss: 0.0014 - PSNR: 29.7022
Epoch 10/50
391/391 [==============================] - 111s 283ms/step - loss: 0.0014 - PSN